In [2]:
# Artificial Neural Network


# Part 1 - Data Preprocessing

# Importing the libraries
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

## Concatenate the Data Frames

X=pd.concat([X,geography,gender],axis=1)

## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [3]:
# Convert to PyTorch tensors
import torch
import torch.nn as nn
import torch.optim as optim

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Part 2 - Now let's make the ANN!

# Define the Neural Network model
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(11, 6)
        self.fc2 = nn.Linear(6, 6)
        self.fc3 = nn.Linear(6, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize the model
model = ANN()

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Part 3 - Training the model

# Number of epochs
epochs = 100

# Training loop
for epoch in range(epochs):
    # Forward pass
    y_pred = model(X_train_tensor)

    # Compute loss
    loss = criterion(y_pred, y_train_tensor.view(-1, 1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Part 4 - Evaluate the model

# Switch to evaluation mode
model.eval()

# Predictions on the test set
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

# Convert predictions to binary (0 or 1)
y_pred_binary = (y_pred_tensor > 0.5).float()

# Convert predictions to NumPy arrays
y_pred_np = y_pred_binary.numpy().flatten()
y_test_np = y_test_tensor.numpy()

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_np, y_pred_np)

# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred_np, y_test_np)

# Print the confusion matrix and accuracy
print("Confusion Matrix:")
print(cm)
print("Accuracy:", score)

ValueError: could not determine the shape of object type 'Series'